In [92]:
import cobra
from projection_methods import FbaProjection, FbaProjectionLowMidConfidence, FbaProjectionHighMidConfidence
import torch
import numpy as np

# A single linear pathway
<table><tr><td>
<img src="models/single_pathway.png">
</td><td> 
        $$
        S=\begin{array}{cc}
        & \begin{array}{ccc} r1 & r2 & r3 \end{array} \\
        \begin{array}{c} \text{A} \\ \text{B} \end{array} &
        \left(
            \begin{array}{ccc}
            1 & -1 & 0 \\
            0 & 1 & -1 \\
            \end{array}
        \right)
        \end{array}
        $$
</td></tr></table>


In [55]:
model = cobra.io.read_sbml_model("models/single_pathway.xml")
S = cobra.util.create_stoichiometric_matrix(model)
fbapro = FbaProjection(stoichiometric_matrix=torch.tensor(S), device=torch.device('cpu'))

No objective coefficients in model. Unclear what should be optimized


## Denoising a given flux vector

In [56]:
noisy_input = torch.tensor([[1, 1, 1]], device=torch.device("cpu"), dtype=torch.float) # input expected as a samples X reactions matrix
# a steady-state flux is of the form (x, x, x). The input is in steady-state, so is returned as is.
fbapro(noisy_input)

tensor([[1.0000, 1.0000, 1.0000]])

In [57]:
noisy_input = torch.tensor([[1.9, 2, 2.1]], device=torch.device("cpu"), dtype=torch.float) # input expected as a samples X reactions matrix
# FBApro finds the closest steady state flux to (1.9, 2, 2.1), which is (2, 2, 2)
fbapro(noisy_input)

tensor([[2.0000, 2.0000, 2.0000]])

In [33]:
noisy_input = torch.tensor([1, 2, 0], device=torch.device("cpu"), dtype=torch.float)
fbapro(noisy_input)

tensor([1.0000, 1.0000, 1.0000])

In [34]:
noisy_input = torch.tensor([1, 0, 1], device=torch.device("cpu"), dtype=torch.float)
fbapro(noisy_input)

tensor([0.6667, 0.6667, 0.6667])

## Imputation and fixed reactions

In [38]:
"""
In the previous example, FBApro treats the 0 as a value indicating the reaction A-->B has no flux, then weighs it against the given fluxes for the other reactions to lower the overall flux through the pathway. 

FBAproPartial gets a list of unknown indices as an input, and finds the closest steady-state flux ignoring flux values in these indices in the distance considered.
""" 
noisy_input = torch.tensor([[1.9, 2, 2.1],
                            [1, 2, 0],
                            [1, 0, 1]], device=torch.device("cpu"), dtype=torch.float)
unknown_indices = [1] # zero based indexing
fbapro_partial = FbaProjectionLowMidConfidence(stoichiometric_matrix=torch.tensor(S), unknown_indices=unknown_indices, device=torch.device('cpu'))
fbapro_partial(noisy_input)

tensor([[2.0000, 2.0000, 2.0000],
        [0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000]])

In [44]:
"""
Consider a scenario in which the last measurement is measured with high fidelity, whereas the others have fluxes estimated from noisy sources.

FBAproFixed receives the indices of the high-confidence reactions, and finds the closest steady-state flux while keeping these reaction fluxes fixed.

Note that in the case of a linear pathway, fixing any single reaction value determines a single steady-state flux respecting it. 
""" 
# noisy_input = torch.tensor([[1.9, 2, 2.1],
#                             [1, 2, 0],
#                             [1, 0, 1]], device=torch.device("cpu"), dtype=torch.float)
measured_indices = [2] # zero based indexing
fbapro_fixed = FbaProjectionHighMidConfidence(stoichiometric_matrix=torch.tensor(S), measured_indices=measured_indices, device=torch.device('cpu'))
fbapro_fixed(noisy_input)

input x - abs min: 0.00e+00, abs max: 2.10e+00, abs mean: 1.22e+00
DDpinv * x - abs min: 0.00e+00, abs max: 2.10e+00, abs mean: 1.22e+00
Suppressing further intersection warnings.


C:\Users\Ariel\PycharmProjects\FBApro\projection_methods.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('stoic', torch.tensor(stoichiometric_matrix, device=device, dtype=dtype),


tensor([[ 2.1000e+00,  2.1000e+00,  2.1000e+00],
        [-5.9605e-08, -5.9605e-08, -5.9605e-08],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00]])

In [49]:
"""
 FBAproFixed is more numerically sensitive, as can be seen from the previous example. When using it to constrain reactions to carry zero flux, it may be easier to remove these from the model temporarily. 
 
 The numerical sensitivity of FBAproFixed can be tuned using the acond and rcond parameters, which set the absolute and relative condition number thresholds for the internal SVD computations.
 """
# noisy_input = torch.tensor([[1.9, 2, 2.1],
#                             [1, 2, 0],
#                             [1, 0, 1]], device=torch.device("cpu"), dtype=torch.float)
measured_indices = [2] # zero based indexing
fbapro_fixed = FbaProjectionHighMidConfidence(stoichiometric_matrix=torch.tensor(S), measured_indices=measured_indices, device=torch.device('cpu'), 
                                              acond=1e-2, rcond=1e-2)
fbapro_fixed(noisy_input)

input x - abs min: 0.00e+00, abs max: 2.10e+00, abs mean: 1.22e+00
DDpinv * x - abs min: 0.00e+00, abs max: 2.10e+00, abs mean: 1.22e+00
Suppressing further intersection warnings.


tensor([[ 2.1000e+00,  2.1000e+00,  2.1000e+00],
        [-5.9605e-08, -5.9605e-08, -5.9605e-08],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00]])

In [52]:
"""
With FBAproFixed it's also possible to provide infeasible inputs. In this linear pathway case, if two reactions are treated as measured but have different fluxes, there is no steady-state flux agreeing with the input. FBAproFixed will attempt to find an appropriate steady-state flux and is expected to behave as FBAproPartial ignoring the non-measured reactions, yet its behavior in this case is not guarenteed.
"""
# noisy_input = torch.tensor([[1.9, 2, 2.1],
#                             [1, 2, 0],
#                             [1, 0, 1]], device=torch.device("cpu"), dtype=torch.float)
measured_indices = [0, 2] # zero based indexing
fbapro_fixed = FbaProjectionHighMidConfidence(stoichiometric_matrix=torch.tensor(S), measured_indices=measured_indices, device=torch.device('cpu'))
fbapro_fixed(noisy_input)

input x - abs min: 0.00e+00, abs max: 2.10e+00, abs mean: 1.22e+00
DDpinv * x - abs min: 2.38e-07, abs max: 2.00e+00, abs mean: 1.22e+00
Suppressing further intersection warnings.


tensor([[2.0000, 2.0000, 2.0000],
        [0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000]])

# A branching pathway
<table><tr><td>
<img src="models/branching_pathway.png">
</td><td> 
        $$
        S=\begin{array}{cc}
        & \begin{array}{ccc} r1 & r2 & r3 & r4 & r5 \end{array} \\
        \begin{array}{c} \text{A} \\\text{Right} \\ \text{Left} \end{array} &
        \left(
            \begin{array}{ccc}
            1 & -1 & 0 & -1 & 0 \\
            0 & 1 & -1 & 0 & 0\\
            0 & 0 & 0 & 1 & -1\\
            \end{array}
        \right)
        \end{array}
        $$
</td></tr></table>


In [72]:
model = cobra.io.read_sbml_model("models/branching_pathway.xml")
S = cobra.util.create_stoichiometric_matrix(model)
fbapro = FbaProjection(stoichiometric_matrix=torch.tensor(S), device=torch.device('cpu'))

No objective coefficients in model. Unclear what should be optimized


In [73]:
"""
Note that here, any three reactions, and any two reactions not in the same branch, are sufficient to define the entire fluxes. There are two degrees of freedom in the model, with fluxes of the form [a+b, a, a, b, b].
"""

noisy_input = torch.tensor([[5, 2, 2, 3, 3],
                            [5, 2, 2.5, 3, 2.5],
                            [10, 2, 2.5, 3, 2.5],
                            [5, 0, 0, 3, 3],
                            [5, 0, 0, 5, 5]], device=torch.device("cpu"), dtype=torch.float) # input expected as a samples X reactions matrix
fbapro(noisy_input)

tensor([[ 5.0000e+00,  2.0000e+00,  2.0000e+00,  3.0000e+00,  3.0000e+00],
        [ 5.0000e+00,  2.2500e+00,  2.2500e+00,  2.7500e+00,  2.7500e+00],
        [ 7.5000e+00,  3.5000e+00,  3.5000e+00,  4.0000e+00,  4.0000e+00],
        [ 4.0000e+00,  5.0000e-01,  5.0000e-01,  3.5000e+00,  3.5000e+00],
        [ 5.0000e+00, -2.2352e-08, -1.4901e-08,  5.0000e+00,  5.0000e+00]])

In [118]:
"""
FBAproPartial can ignore the given left branch fluxes if considered unmeasured (up to small numerical imprecision). The fluxes for that branch are completely determined by the other fluxes, where here any disagreement between A-->Right and Right--> is resolved using the closest fluxes where they agree.
"""
# noisy_input = torch.tensor([[5, 2, 2, 3, 3],
#                             [5, 2, 2.5, 3, 2.5],
#                             [10, 2, 2.5, 3, 2.5],
#                             [5, 0, 0, 3, 3],
#                             [5, 0, 0, 5, 5]], device=torch.device("cpu"), dtype=torch.float) 
partial_output = fbapro_partial(noisy_input)
print(noisy_input)

output - abs min: 8.94e-07, abs max: 1.00e+01, abs mean: 3.60e+00
S * output - abs min: 0.00e+00, abs max: 9.54e-07, abs mean: 6.36e-08
Suppressing further warnings.
tensor([[ 5.0000,  2.0000,  2.0000,  3.0000,  3.0000],
        [ 5.0000,  2.0000,  2.5000,  3.0000,  2.5000],
        [10.0000,  2.0000,  2.5000,  3.0000,  2.5000],
        [ 5.0000,  0.0000,  0.0000,  3.0000,  3.0000],
        [ 5.0000,  0.0000,  0.0000,  5.0000,  5.0000]])
tensor([[ 5.0000e+00,  2.0000e+00,  2.0000e+00,  3.0000e+00,  3.0000e+00],
        [ 5.0000e+00,  2.2500e+00,  2.2500e+00,  2.7500e+00,  2.7500e+00],
        [ 1.0000e+01,  7.2500e+00,  7.2500e+00,  2.7500e+00,  2.7500e+00],
        [ 5.0000e+00,  2.0000e+00,  2.0000e+00,  3.0000e+00,  3.0000e+00],
        [ 5.0000e+00, -8.9407e-07, -8.9407e-07,  5.0000e+00,  5.0000e+00]])


In [121]:
"""
Specifying the upper and right branches as exact measurements for FBAproFixed, since these determine a single steady-state flux, results in the same outputs.
"""
# noisy_input = torch.tensor([[5, 2, 2, 3, 3],
#                             [5, 2, 2.5, 3, 2.5],
#                             [10, 2, 2.5, 3, 2.5],
#                             [5, 0, 0, 3, 3],
#                             [5, 0, 0, 5, 5]], device=torch.device("cpu"), dtype=torch.float) 
fbapro_fixed = FbaProjectionHighMidConfidence(stoichiometric_matrix=torch.tensor(S), measured_indices=[0, 3, 4], device=torch.device('cpu'))
fixed_output = fbapro_fixed(noisy_input) 
print(fixed_output)
print(np.isclose(fixed_output, partial_output, atol=1e-5))

input x - abs min: 0.00e+00, abs max: 1.00e+01, abs mean: 3.04e+00
DDpinv * x - abs min: 6.71e-08, abs max: 1.00e+01, abs mean: 3.04e+00
Suppressing further intersection warnings.
output - abs min: 4.77e-07, abs max: 1.00e+01, abs mean: 3.60e+00
S * output - abs min: 0.00e+00, abs max: 1.43e-06, abs mean: 3.81e-07
Suppressing further warnings.
tensor([[ 5.0000e+00,  2.0000e+00,  2.0000e+00,  3.0000e+00,  3.0000e+00],
        [ 5.0000e+00,  2.2500e+00,  2.2500e+00,  2.7500e+00,  2.7500e+00],
        [ 1.0000e+01,  7.2500e+00,  7.2500e+00,  2.7500e+00,  2.7500e+00],
        [ 5.0000e+00,  2.0000e+00,  2.0000e+00,  3.0000e+00,  3.0000e+00],
        [ 5.0000e+00, -4.7684e-07, -4.7684e-07,  5.0000e+00,  5.0000e+00]])
[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]]


In [125]:
"""
By specifying only the upper branch, FBAproFixed will respect that value and only adjust, though not ignore, values through the other branches
"""
# noisy_input = torch.tensor([[5, 2, 2, 3, 3],
#                             [5, 2, 2.5, 3, 2.5],
#                             [10, 2, 2.5, 3, 2.5],
#                             [5, 0, 0, 3, 3],
#                             [5, 0, 0, 5, 5]], device=torch.device("cpu"), dtype=torch.float) 
fbapro_fixed = FbaProjectionHighMidConfidence(stoichiometric_matrix=torch.tensor(S), measured_indices=[0], device=torch.device('cpu'))
fbapro_fixed(noisy_input) 

input x - abs min: 0.00e+00, abs max: 1.00e+01, abs mean: 3.04e+00
DDpinv * x - abs min: 2.33e-08, abs max: 1.00e+01, abs mean: 3.04e+00
Suppressing further intersection warnings.
output - abs min: 7.15e-07, abs max: 1.00e+01, abs mean: 3.60e+00
S * output - abs min: 0.00e+00, abs max: 4.77e-07, abs mean: 1.75e-07
Suppressing further warnings.


tensor([[ 5.0000e+00,  2.0000e+00,  2.0000e+00,  3.0000e+00,  3.0000e+00],
        [ 5.0000e+00,  2.2500e+00,  2.2500e+00,  2.7500e+00,  2.7500e+00],
        [ 1.0000e+01,  4.7500e+00,  4.7500e+00,  5.2500e+00,  5.2500e+00],
        [ 5.0000e+00,  1.0000e+00,  1.0000e+00,  4.0000e+00,  4.0000e+00],
        [ 5.0000e+00, -7.1526e-07, -7.1526e-07,  5.0000e+00,  5.0000e+00]])